This dataset contains 2 files:

all Harry Potter books in txt file format.
all Harry Potter books in txt file format, but i leave most of the special characters like [, "]. (each sentence ends with '|' for easier splitting)
I made a little preprocess on them and:

removed all unnecessary special characters and left in the text only [. ! ?] characters

removed all newline characters (\n)

removed all carriage return (\r) characters

removed all unnecessary text like page number or book title on each page

added white spaces before all special characters to treat them as separate tokens

fixed all faulty words where:
special character [. ! ?] was at the end of the word
special character [. ! ?] was at the beginning of the word
special character [. ! ?] was in the middle of the word

In [ ]:
## rebel large model


In [1]:
from transformers import pipeline

triplet_extractor = pipeline('text2text-generation', model='Babelscape/rebel-large', tokenizer='Babelscape/rebel-large')
# We need to use the tokenizer manually since we need special tokens.
extracted_text = triplet_extractor.tokenizer.batch_decode([triplet_extractor("Punta Cana is a resort town in the municipality of Higuey, in La Altagracia Province, the eastern most province of the Dominican Republic.", return_tensors=True, return_text=False)[0]["generated_token_ids"]])
print(extracted_text[0])
# Function to parse the generated text and extract the triplets
def extract_triplets(text):
    triplets = []
    relation, subject, relation, object_ = '', '', '', ''
    text = text.strip()
    current = 'x'
    for token in text.replace("<s>", "").replace("<pad>", "").replace("</s>", "").split():
        if token == "<triplet>":
            current = 't'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
                relation = ''
            subject = ''
        elif token == "<subj>":
            current = 's'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
            object_ = ''
        elif token == "<obj>":
            current = 'o'
            relation = ''
        else:
            if current == 't':
                subject += ' ' + token
            elif current == 's':
                object_ += ' ' + token
            elif current == 'o':
                relation += ' ' + token
    if subject != '' and relation != '' and object_ != '':
        triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
    return triplets
extracted_triplets = extract_triplets(extracted_text[0])
print(extracted_triplets)


2025-04-29 12:45:16.783659: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745930716.976934      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745930717.031154      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/344 [00:00<?, ?B/s]

Device set to use cuda:0


<s><triplet> Punta Cana <subj> La Altagracia Province <obj> located in the administrative territorial entity <subj> Dominican Republic <obj> country <triplet> Higuey <subj> La Altagracia Province <obj> located in the administrative territorial entity <subj> Dominican Republic <obj> country <triplet> La Altagracia Province <subj> Dominican Republic <obj> country <triplet> Dominican Republic <subj> La Altagracia Province <obj> contains administrative territorial entity</s>
[{'head': 'Punta Cana', 'type': 'located in the administrative territorial entity', 'tail': 'La Altagracia Province'}, {'head': 'Punta Cana', 'type': 'country', 'tail': 'Dominican Republic'}, {'head': 'Higuey', 'type': 'located in the administrative territorial entity', 'tail': 'La Altagracia Province'}, {'head': 'Higuey', 'type': 'country', 'tail': 'Dominican Republic'}, {'head': 'La Altagracia Province', 'type': 'country', 'tail': 'Dominican Republic'}, {'head': 'Dominican Republic', 'type': 'contains administrative 

In [2]:
def extract_sentences(file_path):
    """
    Reads a text file where sentences are separated by '|'
    and returns a list of cleaned sentences.
    """
    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read()
    # Split on '|' and strip whitespace
    parts = text.split('|')
    # Filter out any empty strings and strip leading/trailing whitespace
    sentences = [p.strip() for p in parts if p.strip()]
    return sentences


sentences = extract_sentences('/kaggle/input/harry-potter-lstm/Harry_Potter_all_char_separated.txt')

In [3]:
sentences[1]

'Mr and Mrs Dursley , of number four , Privet Drive , were proud to say that they were perfectly normal , thank you very much .'

In [4]:
for triplet in extracted_triplets:
    print(triplet)

{'head': 'Punta Cana', 'type': 'located in the administrative territorial entity', 'tail': 'La Altagracia Province'}
{'head': 'Punta Cana', 'type': 'country', 'tail': 'Dominican Republic'}
{'head': 'Higuey', 'type': 'located in the administrative territorial entity', 'tail': 'La Altagracia Province'}
{'head': 'Higuey', 'type': 'country', 'tail': 'Dominican Republic'}
{'head': 'La Altagracia Province', 'type': 'country', 'tail': 'Dominican Republic'}
{'head': 'Dominican Republic', 'type': 'contains administrative territorial entity', 'tail': 'La Altagracia Province'}


In [6]:
import pandas as pd


In [7]:
len(sentences)

79730

In [ ]:
all_triplets = []
i = 47999
sentences = sentences[i:]
for curr_sentence in sentences:
## add loop to this part of this code
    extracted_text = triplet_extractor.tokenizer.batch_decode([triplet_extractor(curr_sentence, return_tensors=True, return_text=False)[0]["generated_token_ids"]])
    extracted_triplets = extract_triplets(extracted_text[0])
    for triplet in extracted_triplets:
        all_triplets.append(triplet)
    if(i%100==0):
        print(f"{i}th sentence has {len(extracted_triplets)}")
    if(i%1000==0):
        df = pd.DataFrame(all_triplets, columns=['head', 'type', 'tail'])
        df.to_csv(f'output_{i}.csv', index=False, encoding='utf-8')
        print("Wrote", len(df), "rows to output.csv")
    i= i + 1

df = pd.DataFrame(all_triplets, columns=['head', 'type', 'tail'])
df.to_csv(f'output_{i}_final.csv', index=False, encoding='utf-8')
print("Wrote", len(df), "rows to output_final.csv")

48000th sentence has 1
Wrote 3 rows to output.csv
48100th sentence has 2
48200th sentence has 2
48300th sentence has 2
48400th sentence has 1
48500th sentence has 1
48600th sentence has 2
48700th sentence has 2
48800th sentence has 1
48900th sentence has 9
49000th sentence has 1
Wrote 1997 rows to output.csv
49100th sentence has 1
49200th sentence has 2
49300th sentence has 2
49400th sentence has 2
49500th sentence has 2
49600th sentence has 2
49700th sentence has 2
49800th sentence has 2
49900th sentence has 2
50000th sentence has 2
Wrote 3891 rows to output.csv
50100th sentence has 2
50200th sentence has 2
50300th sentence has 2
50400th sentence has 1
50500th sentence has 1
50600th sentence has 1
50700th sentence has 1
50800th sentence has 1
50900th sentence has 2
51000th sentence has 1
Wrote 5913 rows to output.csv
51100th sentence has 2
51200th sentence has 2
51300th sentence has 2
51400th sentence has 2
51500th sentence has 2
51600th sentence has 2
51700th sentence has 1
51800th s

In [ ]:

df = pd.DataFrame(all_triplets, columns=['head', 'type', 'tail'])
df.to_csv(f'final_output_{i}.csv', index=False, encoding='utf-8')
print("Wrote", len(df), "rows to output.csv")